# Домашнее задание

1. Попробуйте обучить нейронную сеть GRU/LSTM для предсказания сентимента сообщений с твитера на примере https://www.kaggle.com/datasets/arkhoshghalb/twitter-sentiment-analysis-hatred-speech

2. Опишите, какой результат вы получили? Что помогло вам улучшить ее точность?

Подключаемся к google disc

In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Распаковываем архив

In [2]:
! unzip '/content/drive/MyDrive/Twitter Sentiment Analysis.zip'

Archive:  /content/drive/MyDrive/Twitter Sentiment Analysis.zip
  inflating: test.csv                
  inflating: train.csv               


Смотрим на данные

In [3]:
import pandas as pd

df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [4]:
df_train[df_train['label']==0].head(5)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
df_train[df_train['label']==1].head(7)

,id,label,tweet
13,14,1,@user #cnn calls #michigan middle school 'buil...
14,15,1,no comment! in #australia #opkillingbay #se...
17,18,1,retweet if you agree!
23,24,1,@user @user lumpy says i am a . prove it lumpy.
34,35,1,it's unbelievable that in the 21st century we'...
56,57,1,@user lets fight against #love #peace
68,69,1,ð©the white establishment can't have blk fol...


0 - positive<br>
1 - negative

In [6]:
df_train['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [7]:
df_test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


##Предобработка

In [8]:
!pip install stop-words pymorphy2

     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 8.2 MB 29.3 MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=87b0db533ee6f17185e854ed4e9fda3250319ca4abdae325d89796746da168b7
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [9]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

Сохраняем в переменные стоп-слова и знаки пунктуации

In [10]:
sw = set(get_stop_words("en"))
#sw

In [11]:
puncts = set(punctuation)

Создаем функцию по очистке твитов от стоп-слов, знаков пунктуации приводим всё в нижний регистр

In [12]:
morpher = MorphAnalyzer()


def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in puncts)
    txt = txt.lower()
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [13]:
print(df_train['tweet'].iloc[:1])

0     @user when a father is dysfunctional and is s...
Name: tweet, dtype: object


In [14]:
df_train['tweet'].iloc[:1].apply(preprocess_text).values

array(['user father dysfunctional selfish drags kids dysfunction run'],
      dtype=object)

Разбиваем обучающую выборку на трейн и валидацию

In [15]:
from sklearn.model_selection import train_test_split

def train_valid_split(Xt):
    df_train, df_val = train_test_split(Xt, test_size=0.15, random_state=13)
    return df_train, df_val

In [16]:
df_train, df_val = train_valid_split(df_train)

In [17]:
df_train['label'].value_counts()

0    25250
1     1917
Name: label, dtype: int64

In [18]:
df_val['label'].value_counts()

0    4470
1     325
Name: label, dtype: int64

Структура лейблов в тренировочной и валидационной выборке примерно одинакова

Делаем препроцессинг текстов в трейне и валидации

In [19]:
from tqdm import tqdm 
tqdm.pandas()

df_train['tweet'] = df_train['tweet'].progress_apply(preprocess_text)
df_val['tweet'] = df_val['tweet'].progress_apply(preprocess_text)

100%|██████████| 4795/4795 [00:01<00:00, 3956.40it/s]


Создаем корпус слов, на котором будем обучаться

In [20]:
train_corpus = " ".join(df_train["tweet"])
train_corpus = train_corpus.lower()
#train_corpus

Разбиваем весь корпус на токены

In [21]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)
tokens[:5]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['thankful', 'sense', 'touch', 'thankful', 'positive']

Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [22]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [23]:
from nltk.probability import FreqDist

max_words = 2500
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]  # вычитание 1 для padding
len(tokens_filtered_top)

2499

In [24]:
tokens_filtered_top[:10]

['user', 'love', 'day', 'happy', 'amp', 'just', 'will', 'im', 'u', 'time']

Соберем словарь из самых популярных токенов, где каждому токену будет присвоен уникальный индекс

In [25]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
#vocabulary

Создадим функцию по созданию из твитов последовательностей заданной длины

In [26]:
import numpy as np
max_len = 20

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen-len(result))
    return result[-maxlen:] + padding

Переведем трейн и валидационную выборку в набор последовательностей

In [27]:
%%time
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["tweet"]])
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["tweet"]])

CPU times: user 3.86 s, sys: 14.3 ms, total: 3.87 s
Wall time: 3.87 s


In [28]:
df_train['tweet'].iloc[0]

'thankful sense touch thankful positive'

In [29]:
x_train[0]

array([  16,  895, 1406,   16,   18,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

Создаем архитектуру нейронной сети LSTM

In [114]:
import torch
import torch.nn as nn
import torch.nn.functional as F


from torch import nn


class LSTMFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, ht = self.lstm(x)
       
        if self.use_last:
            last_tensor = lstm_out[:,-1,:]
        else:
            # use mean
            last_tensor = torch.mean(lstm_out[:,:], dim=1)
    
        out = self.linear(last_tensor)
        # print(out.shape)
        return torch.sigmoid(out)

Создаем кастомный класс Датасета

In [115]:
from torch.utils.data import DataLoader, Dataset


class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

Создаем из наших данных кастомные датасеты и создаем лоадеры

In [116]:
batch_size = 128

train_dataset = DataWrapper(x_train, df_train['label'].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = DataWrapper(x_val, df_val['label'].values)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

Создаем модель

In [117]:
model = LSTMFixedLen(vocab_size=max_words)
device = 'cpu'

In [118]:
print(model)
print("Parameters:", sum([param.nelement() for param in model.parameters()]))

LSTMFixedLen(
  (embeddings): Embedding(2500, 128, padding_idx=0)
  (lstm): LSTM(128, 128, num_layers=2, batch_first=True)
  (linear): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Parameters: 584321


Функция ошибки и оптимизатор

In [119]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

# Т.к. имеется сильный дисбаланс классов, для логирования я использую F-меру для минорного класса (которого значительно меньше)

Запускаем обучение

In [120]:
from ssl import PROTOCOL_TLS_CLIENT
model = model.to(device)
model.train()
th = 0.5
epochs = 5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    tp, fp, fn = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()

        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        tp += ((labels==1) & (pred_labels == 1)).sum()
        fp += ((labels==0) & (pred_labels == 1)).sum()
        fn += ((labels==1) & (pred_labels == 0)).sum()
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f_measure = 2*(precision*recall)/ (precision + recall)
        #tp_running_items += len([l for l in labels if l==1])       
        # выводим статистику о процессе обучения
        if i % 150 == 0:    # печатаем каждые 150 batches
            model.eval()
            
            print(f'Epoch [{epoch + 1}/{epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {loss:.3f}. ' \
                  #f'Precision: {precision:.3f}. ' \
                  #f'Recall: {recall:.3f}. ' \
                  f'F-measure: {f_measure:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0
            train_loss_history.append(loss)

            # выводим статистику на тестовых данных
            test_tp, test_fp, test_fn = 0.0, 0.0, 0.0
            for j, data in enumerate(val_loader):
                test_labels = data[1].to(device)
                test_outputs = model(data[0].to(device))
                
                # подсчет ошибки на тесте
                test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
                # подсчет метрики на тесте

                pred_test_labels = torch.squeeze((test_outputs > th).int())
                test_tp += ((test_labels==1) & (pred_test_labels == 1)).sum()
                test_fp += ((test_labels==0) & (pred_test_labels == 1)).sum()
                test_fn += ((test_labels==1) & (pred_test_labels == 0)).sum()
                test_precision = test_tp / (test_tp + test_fp)
                test_recall = test_tp / (test_tp + test_fn)
                test_f_measure = 2*(test_precision*test_recall)/ (test_precision + test_recall)

            test_loss_history.append(test_loss.item())
            print(f'Test loss: {test_loss:.3f}.' \
                  #f'Test Precision: {test_recall:.3f}. ' \
                  #f'Test Recall: {test_recall:.3f}. ' \
                  f'Test F-measure: {test_f_measure:.3f}', end='. ')
            print('\n')
        
        model.train()
        
print('Training is finished!')

Epoch [1/5]. Step [1/213]. Loss: 0.721. F-measure: 0.184. Test loss: 0.644.Test F-measure: nan. 

Epoch [1/5]. Step [151/213]. Loss: 0.196. F-measure: 0.017. Test loss: 0.074.Test F-measure: nan. 

Epoch [2/5]. Step [1/213]. Loss: 0.295. F-measure: nan. Test loss: 0.068.Test F-measure: nan. 

Epoch [2/5]. Step [151/213]. Loss: 0.302. F-measure: nan. Test loss: 0.040.Test F-measure: nan. 

Epoch [3/5]. Step [1/213]. Loss: 0.243. F-measure: nan. Test loss: 0.084.Test F-measure: nan. 

Epoch [3/5]. Step [151/213]. Loss: 0.187. F-measure: nan. Test loss: 0.047.Test F-measure: nan. 

Epoch [4/5]. Step [1/213]. Loss: 0.242. F-measure: nan. Test loss: 0.054.Test F-measure: nan. 

Epoch [4/5]. Step [151/213]. Loss: 0.063. F-measure: 0.201. Test loss: 0.996.Test F-measure: 0.012. 

Epoch [5/5]. Step [1/213]. Loss: 0.183. F-measure: 0.609. Test loss: 0.096.Test F-measure: 0.507. 

Epoch [5/5]. Step [151/213]. Loss: 0.143. F-measure: 0.604. Test loss: 0.114.Test F-measure: 0.522. 

Training is fi

In [122]:
lstm_2_layers_test_f_measure = test_f_measure
lstm_2_layers_train_f_measure = f_measure
lstm_2_layers_test_f_measure, lstm_2_layers_train_f_measure

(tensor(0.5216), tensor(0.5975))

Сначала модель все предсказывала нулем, но к 4-й эпохе f-мера по 1 классу (которого гораздо меньше), наконец, начала расти. Итоговый результат не самый лучший, вероятно еще стоит обучить 5 эпох




In [123]:
model = model.to(device)
model.train()
th = 0.5
epochs = 5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    tp, fp, fn = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()

        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        tp += ((labels==1) & (pred_labels == 1)).sum()
        fp += ((labels==0) & (pred_labels == 1)).sum()
        fn += ((labels==1) & (pred_labels == 0)).sum()
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f_measure = 2*(precision*recall)/ (precision + recall)
        #tp_running_items += len([l for l in labels if l==1])       
        # выводим статистику о процессе обучения
        if i % 150 == 0:    # печатаем каждые 150 batches
            model.eval()
            
            print(f'Epoch [{epoch + 1}/{epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {loss:.3f}. ' \
                  #f'Precision: {precision:.3f}. ' \
                  #f'Recall: {recall:.3f}. ' \
                  f'F-measure: {f_measure:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0
            train_loss_history.append(loss)

            # выводим статистику на тестовых данных
            test_tp, test_fp, test_fn = 0.0, 0.0, 0.0
            for j, data in enumerate(val_loader):
                test_labels = data[1].to(device)
                test_outputs = model(data[0].to(device))
                
                # подсчет ошибки на тесте
                test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
                # подсчет метрики на тесте

                pred_test_labels = torch.squeeze((test_outputs > th).int())
                test_tp += ((test_labels==1) & (pred_test_labels == 1)).sum()
                test_fp += ((test_labels==0) & (pred_test_labels == 1)).sum()
                test_fn += ((test_labels==1) & (pred_test_labels == 0)).sum()
                test_precision = test_tp / (test_tp + test_fp)
                test_recall = test_tp / (test_tp + test_fn)
                test_f_measure = 2*(test_precision*test_recall)/ (test_precision + test_recall)

            test_loss_history.append(test_loss.item())
            print(f'Test loss: {test_loss:.3f}.' \
                  #f'Test Precision: {test_recall:.3f}. ' \
                  #f'Test Recall: {test_recall:.3f}. ' \
                  f'Test F-measure: {test_f_measure:.3f}', end='. ')
            print('\n')
        
        model.train()
        
print('Training is finished!')

Epoch [1/5]. Step [1/213]. Loss: 0.107. F-measure: 0.667. Test loss: 0.008.Test F-measure: 0.457. 

Epoch [1/5]. Step [151/213]. Loss: 0.114. F-measure: 0.648. Test loss: 0.005.Test F-measure: 0.410. 

Epoch [2/5]. Step [1/213]. Loss: 0.069. F-measure: 0.800. Test loss: 0.154.Test F-measure: 0.593. 

Epoch [2/5]. Step [151/213]. Loss: 0.192. F-measure: 0.701. Test loss: 1.980.Test F-measure: 0.566. 

Epoch [3/5]. Step [1/213]. Loss: 0.133. F-measure: 0.828. Test loss: 0.005.Test F-measure: 0.588. 

Epoch [3/5]. Step [151/213]. Loss: 0.107. F-measure: 0.693. Test loss: 0.032.Test F-measure: 0.569. 

Epoch [4/5]. Step [1/213]. Loss: 0.197. F-measure: 0.714. Test loss: 0.190.Test F-measure: 0.590. 

Epoch [4/5]. Step [151/213]. Loss: 0.060. F-measure: 0.711. Test loss: 0.027.Test F-measure: 0.540. 

Epoch [5/5]. Step [1/213]. Loss: 0.069. F-measure: 0.917. Test loss: 0.005.Test F-measure: 0.587. 

Epoch [5/5]. Step [151/213]. Loss: 0.094. F-measure: 0.738. Test loss: 0.007.Test F-measure:

In [124]:
lstm_2_layers_test_f_measure_10ep = test_f_measure
lstm_2_layers_train_f_measure_10ep = f_measure
lstm_2_layers_test_f_measure_10ep, lstm_2_layers_train_f_measure_10ep

(tensor(0.5582), tensor(0.7362))

после дополнительных 5-ти эпох метрика увеличилась, но очень уж большое переобучение

## Попробуем LSTM из 1-го слоя

In [135]:
class LSTMFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=1, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, ht = self.lstm(x)
       
        if self.use_last:
            last_tensor = lstm_out[:,-1,:]
        else:
            # use mean
            last_tensor = torch.mean(lstm_out[:,:], dim=1)
    
        out = self.linear(last_tensor)
        # print(out.shape)
        return torch.sigmoid(out)

In [136]:
model = LSTMFixedLen(vocab_size=max_words)
device = 'cpu'

In [137]:
print(model)
print("Parameters:", sum([param.nelement() for param in model.parameters()]))

LSTMFixedLen(
  (embeddings): Embedding(2500, 128, padding_idx=0)
  (lstm): LSTM(128, 128, batch_first=True)
  (linear): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Parameters: 452225


In [138]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [139]:
model = model.to(device)
model.train()
th = 0.5
epochs = 5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    tp, fp, fn = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()

        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        tp += ((labels==1) & (pred_labels == 1)).sum()
        fp += ((labels==0) & (pred_labels == 1)).sum()
        fn += ((labels==1) & (pred_labels == 0)).sum()
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f_measure = 2*(precision*recall)/ (precision + recall)
        #tp_running_items += len([l for l in labels if l==1])       
        # выводим статистику о процессе обучения
        if i % 150 == 0:    # печатаем каждые 150 batches
            model.eval()
            
            print(f'Epoch [{epoch + 1}/{epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {loss:.3f}. ' \
                  #f'Precision: {precision:.3f}. ' \
                  #f'Recall: {recall:.3f}. ' \
                  f'F-measure: {f_measure:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0
            train_loss_history.append(loss)

            # выводим статистику на тестовых данных
            test_tp, test_fp, test_fn = 0.0, 0.0, 0.0
            for j, data in enumerate(val_loader):
                test_labels = data[1].to(device)
                test_outputs = model(data[0].to(device))
                
                # подсчет ошибки на тесте
                test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
                # подсчет метрики на тесте

                pred_test_labels = torch.squeeze((test_outputs > th).int())
                test_tp += ((test_labels==1) & (pred_test_labels == 1)).sum()
                test_fp += ((test_labels==0) & (pred_test_labels == 1)).sum()
                test_fn += ((test_labels==1) & (pred_test_labels == 0)).sum()
                test_precision = test_tp / (test_tp + test_fp)
                test_recall = test_tp / (test_tp + test_fn)
                test_f_measure = 2*(test_precision*test_recall)/ (test_precision + test_recall)

            test_loss_history.append(test_loss.item())
            print(f'Test loss: {test_loss:.3f}.' \
                  #f'Test Precision: {test_recall:.3f}. ' \
                  #f'Test Recall: {test_recall:.3f}. ' \
                  f'Test F-measure: {test_f_measure:.3f}', end='. ')
            print('\n')
        
        model.train()
        
print('Training is finished!')

Epoch [1/5]. Step [1/213]. Loss: 0.730. F-measure: 0.158. Test loss: 0.624.Test F-measure: nan. 

Epoch [1/5]. Step [151/213]. Loss: 0.260. F-measure: 0.016. Test loss: 0.945.Test F-measure: nan. 

Epoch [2/5]. Step [1/213]. Loss: 0.273. F-measure: nan. Test loss: 0.857.Test F-measure: nan. 

Epoch [2/5]. Step [151/213]. Loss: 0.320. F-measure: nan. Test loss: 0.064.Test F-measure: nan. 

Epoch [3/5]. Step [1/213]. Loss: 0.259. F-measure: nan. Test loss: 0.078.Test F-measure: nan. 

Epoch [3/5]. Step [151/213]. Loss: 0.273. F-measure: nan. Test loss: 0.050.Test F-measure: nan. 

Epoch [4/5]. Step [1/213]. Loss: 0.225. F-measure: nan. Test loss: 0.078.Test F-measure: nan. 

Epoch [4/5]. Step [151/213]. Loss: 0.164. F-measure: nan. Test loss: 0.676.Test F-measure: nan. 

Epoch [5/5]. Step [1/213]. Loss: 0.314. F-measure: 0.143. Test loss: 0.044.Test F-measure: 0.383. 

Epoch [5/5]. Step [151/213]. Loss: 0.221. F-measure: 0.478. Test loss: 0.008.Test F-measure: 0.467. 

Training is finish

In [140]:
lstm_1_layer_test_f_measure = test_f_measure
lstm_1_layer_train_f_measure = f_measure
lstm_1_layer_test_f_measure, lstm_1_layer_train_f_measure

(tensor(0.4665), tensor(0.4873))

Однослойная показала результат хуже 2-слойной

Попробуем 3 слоя

In [141]:
class LSTMFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=256, hidden_dim=128, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=3, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, ht = self.lstm(x)
       
        if self.use_last:
            last_tensor = lstm_out[:,-1,:]
        else:
            # use mean
            last_tensor = torch.mean(lstm_out[:,:], dim=1)
    
        out = self.linear(last_tensor)
        # print(out.shape)
        return torch.sigmoid(out)

In [142]:
model = LSTMFixedLen(vocab_size=max_words)
device = 'cpu'

In [143]:
print(model)
print("Parameters:", sum([param.nelement() for param in model.parameters()]))

LSTMFixedLen(
  (embeddings): Embedding(2500, 256, padding_idx=0)
  (lstm): LSTM(256, 128, num_layers=3, batch_first=True)
  (linear): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Parameters: 1101953


In [144]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [145]:
model = model.to(device)
model.train()
th = 0.5
epochs = 5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    tp, fp, fn = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()

        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        tp += ((labels==1) & (pred_labels == 1)).sum()
        fp += ((labels==0) & (pred_labels == 1)).sum()
        fn += ((labels==1) & (pred_labels == 0)).sum()
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f_measure = 2*(precision*recall)/ (precision + recall)
        #tp_running_items += len([l for l in labels if l==1])       
        # выводим статистику о процессе обучения
        if i % 150 == 0:    # печатаем каждые 150 batches
            model.eval()
            
            print(f'Epoch [{epoch + 1}/{epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {loss:.3f}. ' \
                  #f'Precision: {precision:.3f}. ' \
                  #f'Recall: {recall:.3f}. ' \
                  f'F-measure: {f_measure:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0
            train_loss_history.append(loss)

            # выводим статистику на тестовых данных
            test_tp, test_fp, test_fn = 0.0, 0.0, 0.0
            for j, data in enumerate(val_loader):
                test_labels = data[1].to(device)
                test_outputs = model(data[0].to(device))
                
                # подсчет ошибки на тесте
                test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
                # подсчет метрики на тесте

                pred_test_labels = torch.squeeze((test_outputs > th).int())
                test_tp += ((test_labels==1) & (pred_test_labels == 1)).sum()
                test_fp += ((test_labels==0) & (pred_test_labels == 1)).sum()
                test_fn += ((test_labels==1) & (pred_test_labels == 0)).sum()
                test_precision = test_tp / (test_tp + test_fp)
                test_recall = test_tp / (test_tp + test_fn)
                test_f_measure = 2*(test_precision*test_recall)/ (test_precision + test_recall)

            test_loss_history.append(test_loss.item())
            print(f'Test loss: {test_loss:.3f}.' \
                  #f'Test Precision: {test_recall:.3f}. ' \
                  #f'Test Recall: {test_recall:.3f}. ' \
                  f'Test F-measure: {test_f_measure:.3f}', end='. ')
            print('\n')
        
        model.train()
        
print('Training is finished!')

Epoch [1/5]. Step [1/213]. Loss: 0.718. F-measure: 0.118. Test loss: 0.485.Test F-measure: nan. 

Epoch [1/5]. Step [151/213]. Loss: 0.239. F-measure: 0.011. Test loss: 0.054.Test F-measure: nan. 

Epoch [2/5]. Step [1/213]. Loss: 0.315. F-measure: nan. Test loss: 0.083.Test F-measure: nan. 

Epoch [2/5]. Step [151/213]. Loss: 0.220. F-measure: nan. Test loss: 0.077.Test F-measure: nan. 

Epoch [3/5]. Step [1/213]. Loss: 0.273. F-measure: nan. Test loss: 0.046.Test F-measure: nan. 

Epoch [3/5]. Step [151/213]. Loss: 0.244. F-measure: nan. Test loss: 0.036.Test F-measure: nan. 

Epoch [4/5]. Step [1/213]. Loss: 0.291. F-measure: nan. Test loss: 0.148.Test F-measure: nan. 

Epoch [4/5]. Step [151/213]. Loss: 0.124. F-measure: nan. Test loss: 0.099.Test F-measure: nan. 

Epoch [5/5]. Step [1/213]. Loss: 0.181. F-measure: nan. Test loss: 0.094.Test F-measure: nan. 

Epoch [5/5]. Step [151/213]. Loss: 0.080. F-measure: 0.363. Test loss: 0.443.Test F-measure: 0.357. 

Training is finished!


In [146]:
lstm_3_layers_test_f_measure = test_f_measure
lstm_3_layers_train_f_measure = f_measure
lstm_3_layers_test_f_measure, lstm_3_layers_train_f_measure

(tensor(0.3571), tensor(0.3885))

Слабый результат на 5-ти эпохах и очень долгое обучение

Вернем 1 слой и увеличим размер эмбединга и скрытого расстояния

In [161]:
class LSTMFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=256, hidden_dim=256, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=1, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, ht = self.lstm(x)
       
        if self.use_last:
            last_tensor = lstm_out[:,-1,:]
        else:
            # use mean
            last_tensor = torch.mean(lstm_out[:,:], dim=1)
    
        out = self.linear(last_tensor)
        # print(out.shape)
        return torch.sigmoid(out)

In [162]:
model = LSTMFixedLen(vocab_size=max_words)
device = 'cpu'

In [163]:
print(model)
print("Parameters:", sum([param.nelement() for param in model.parameters()]))

LSTMFixedLen(
  (embeddings): Embedding(2500, 256, padding_idx=0)
  (lstm): LSTM(256, 256, batch_first=True)
  (linear): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Parameters: 1166593


In [164]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [165]:
model = model.to(device)
model.train()
th = 0.5
epochs = 5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    tp, fp, fn = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()

        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        tp += ((labels==1) & (pred_labels == 1)).sum()
        fp += ((labels==0) & (pred_labels == 1)).sum()
        fn += ((labels==1) & (pred_labels == 0)).sum()
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f_measure = 2*(precision*recall)/ (precision + recall)
        #tp_running_items += len([l for l in labels if l==1])       
        # выводим статистику о процессе обучения
        if i % 150 == 0:    # печатаем каждые 150 batches
            model.eval()
            
            print(f'Epoch [{epoch + 1}/{epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {loss:.3f}. ' \
                  #f'Precision: {precision:.3f}. ' \
                  #f'Recall: {recall:.3f}. ' \
                  f'F-measure: {f_measure:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0
            train_loss_history.append(loss)

            # выводим статистику на тестовых данных
            test_tp, test_fp, test_fn = 0.0, 0.0, 0.0
            for j, data in enumerate(val_loader):
                test_labels = data[1].to(device)
                test_outputs = model(data[0].to(device))
                
                # подсчет ошибки на тесте
                test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
                # подсчет метрики на тесте

                pred_test_labels = torch.squeeze((test_outputs > th).int())
                test_tp += ((test_labels==1) & (pred_test_labels == 1)).sum()
                test_fp += ((test_labels==0) & (pred_test_labels == 1)).sum()
                test_fn += ((test_labels==1) & (pred_test_labels == 0)).sum()
                test_precision = test_tp / (test_tp + test_fp)
                test_recall = test_tp / (test_tp + test_fn)
                test_f_measure = 2*(test_precision*test_recall)/ (test_precision + test_recall)

            test_loss_history.append(test_loss.item())
            print(f'Test loss: {test_loss:.3f}.' \
                  #f'Test Precision: {test_recall:.3f}. ' \
                  #f'Test Recall: {test_recall:.3f}. ' \
                  f'Test F-measure: {test_f_measure:.3f}', end='. ')
            print('\n')
        
        model.train()
        
print('Training is finished!')

Epoch [1/5]. Step [1/213]. Loss: 0.704. F-measure: 0.061. Test loss: 0.478.Test F-measure: nan. 

Epoch [1/5]. Step [151/213]. Loss: 0.237. F-measure: 0.005. Test loss: 0.053.Test F-measure: nan. 

Epoch [2/5]. Step [1/213]. Loss: 0.173. F-measure: nan. Test loss: 0.063.Test F-measure: nan. 

Epoch [2/5]. Step [151/213]. Loss: 0.356. F-measure: nan. Test loss: 0.074.Test F-measure: nan. 

Epoch [3/5]. Step [1/213]. Loss: 0.212. F-measure: 0.154. Test loss: 0.042.Test F-measure: 0.269. 

Epoch [3/5]. Step [151/213]. Loss: 0.185. F-measure: 0.352. Test loss: 0.007.Test F-measure: 0.371. 

Epoch [4/5]. Step [1/213]. Loss: 0.175. F-measure: 0.353. Test loss: 0.041.Test F-measure: 0.460. 

Epoch [4/5]. Step [151/213]. Loss: 0.101. F-measure: 0.568. Test loss: 0.039.Test F-measure: 0.408. 

Epoch [5/5]. Step [1/213]. Loss: 0.209. F-measure: 0.560. Test loss: 0.084.Test F-measure: 0.477. 

Epoch [5/5]. Step [151/213]. Loss: 0.128. F-measure: 0.637. Test loss: 0.009.Test F-measure: 0.478. 

Tr

In [166]:
lstm_1_layer_test_f_measure_256_emb = test_f_measure
lstm_1_layer_train_f_measure_256_emb = f_measure
lstm_1_layer_test_f_measure_256_emb, lstm_1_layer_train_f_measure_256_emb

(tensor(0.4777), tensor(0.6437))

Увеличение размерности эмбединга и скрытых состояний дало положительный прирост к метрике, но также есть большое переобучение

Наконец, попробуем обычную 2-слойную LSTM, но будем снимать не послдений скрытый слой, а будем усреднять

In [168]:
class LSTMFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=False):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, ht = self.lstm(x)
       
        if self.use_last:
            last_tensor = lstm_out[:,-1,:]
        else:
            # use mean
            last_tensor = torch.mean(lstm_out[:,:], dim=1)
    
        out = self.linear(last_tensor)
        # print(out.shape)
        return torch.sigmoid(out)

In [169]:
model = LSTMFixedLen(vocab_size=max_words)
device = 'cpu'

In [170]:
print(model)
print("Parameters:", sum([param.nelement() for param in model.parameters()]))

LSTMFixedLen(
  (embeddings): Embedding(2500, 128, padding_idx=0)
  (lstm): LSTM(128, 128, num_layers=2, batch_first=True)
  (linear): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Parameters: 584321


In [171]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [172]:
model = model.to(device)
model.train()
th = 0.5
epochs = 5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    tp, fp, fn = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()

        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        tp += ((labels==1) & (pred_labels == 1)).sum()
        fp += ((labels==0) & (pred_labels == 1)).sum()
        fn += ((labels==1) & (pred_labels == 0)).sum()
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f_measure = 2*(precision*recall)/ (precision + recall)
        #tp_running_items += len([l for l in labels if l==1])       
        # выводим статистику о процессе обучения
        if i % 150 == 0:    # печатаем каждые 150 batches
            model.eval()
            
            print(f'Epoch [{epoch + 1}/{epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {loss:.3f}. ' \
                  #f'Precision: {precision:.3f}. ' \
                  #f'Recall: {recall:.3f}. ' \
                  f'F-measure: {f_measure:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0
            train_loss_history.append(loss)

            # выводим статистику на тестовых данных
            test_tp, test_fp, test_fn = 0.0, 0.0, 0.0
            for j, data in enumerate(val_loader):
                test_labels = data[1].to(device)
                test_outputs = model(data[0].to(device))
                
                # подсчет ошибки на тесте
                test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
                # подсчет метрики на тесте

                pred_test_labels = torch.squeeze((test_outputs > th).int())
                test_tp += ((test_labels==1) & (pred_test_labels == 1)).sum()
                test_fp += ((test_labels==0) & (pred_test_labels == 1)).sum()
                test_fn += ((test_labels==1) & (pred_test_labels == 0)).sum()
                test_precision = test_tp / (test_tp + test_fp)
                test_recall = test_tp / (test_tp + test_fn)
                test_f_measure = 2*(test_precision*test_recall)/ (test_precision + test_recall)

            test_loss_history.append(test_loss.item())
            print(f'Test loss: {test_loss:.3f}.' \
                  #f'Test Precision: {test_recall:.3f}. ' \
                  #f'Test Recall: {test_recall:.3f}. ' \
                  f'Test F-measure: {test_f_measure:.3f}', end='. ')
            print('\n')
        
        model.train()
        
print('Training is finished!')

Epoch [1/5]. Step [1/213]. Loss: 0.702. F-measure: 0.031. Test loss: 0.521.Test F-measure: nan. 

Epoch [1/5]. Step [151/213]. Loss: 0.173. F-measure: 0.315. Test loss: 0.019.Test F-measure: 0.507. 

Epoch [2/5]. Step [1/213]. Loss: 0.085. F-measure: 0.857. Test loss: 0.068.Test F-measure: 0.547. 

Epoch [2/5]. Step [151/213]. Loss: 0.149. F-measure: 0.644. Test loss: 0.037.Test F-measure: 0.584. 

Epoch [3/5]. Step [1/213]. Loss: 0.052. F-measure: 0.833. Test loss: 0.008.Test F-measure: 0.599. 

Epoch [3/5]. Step [151/213]. Loss: 0.067. F-measure: 0.723. Test loss: 0.013.Test F-measure: 0.609. 

Epoch [4/5]. Step [1/213]. Loss: 0.063. F-measure: 0.750. Test loss: 0.007.Test F-measure: 0.631. 

Epoch [4/5]. Step [151/213]. Loss: 0.083. F-measure: 0.768. Test loss: 0.034.Test F-measure: 0.611. 

Epoch [5/5]. Step [1/213]. Loss: 0.078. F-measure: 0.750. Test loss: 0.026.Test F-measure: 0.621. 

Epoch [5/5]. Step [151/213]. Loss: 0.072. F-measure: 0.807. Test loss: 0.780.Test F-measure: 0

In [173]:
lstm_2_layers_test_f_measure_mean_last_tensor = test_f_measure
lstm_2_layers_train_f_measure_mean_last_tensor = f_measure
lstm_2_layers_test_f_measure_mean_last_tensor, lstm_2_layers_train_f_measure_mean_last_tensor

(tensor(0.6106), tensor(0.7991))

Данная архитектура показала лучший результат, но есть переобучение

## Попробуем GRU

2 слоя, снимаем состояние с последнего скрытого состояния

In [174]:
class GRUFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        gru_out, ht = self.gru(x)
       
        if self.use_last:
            last_tensor = gru_out[:,-1,:]
        else:
            # use mean
            last_tensor = torch.mean(gru_out[:,:], dim=1)
    
        out = self.linear(last_tensor)
        return torch.sigmoid(out)

In [175]:
model = LSTMFixedLen(vocab_size=max_words)
device = 'cpu'

In [176]:
print(model)
print("Parameters:", sum([param.nelement() for param in model.parameters()]))

LSTMFixedLen(
  (embeddings): Embedding(2500, 128, padding_idx=0)
  (lstm): LSTM(128, 128, num_layers=2, batch_first=True)
  (linear): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Parameters: 584321


In [177]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [178]:
model = model.to(device)
model.train()
th = 0.5
epochs = 5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    tp, fp, fn = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()

        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        tp += ((labels==1) & (pred_labels == 1)).sum()
        fp += ((labels==0) & (pred_labels == 1)).sum()
        fn += ((labels==1) & (pred_labels == 0)).sum()
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f_measure = 2*(precision*recall)/ (precision + recall)
        #tp_running_items += len([l for l in labels if l==1])       
        # выводим статистику о процессе обучения
        if i % 150 == 0:    # печатаем каждые 150 batches
            model.eval()
            
            print(f'Epoch [{epoch + 1}/{epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {loss:.3f}. ' \
                  #f'Precision: {precision:.3f}. ' \
                  #f'Recall: {recall:.3f}. ' \
                  f'F-measure: {f_measure:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0
            train_loss_history.append(loss)

            # выводим статистику на тестовых данных
            test_tp, test_fp, test_fn = 0.0, 0.0, 0.0
            for j, data in enumerate(val_loader):
                test_labels = data[1].to(device)
                test_outputs = model(data[0].to(device))
                
                # подсчет ошибки на тесте
                test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
                # подсчет метрики на тесте

                pred_test_labels = torch.squeeze((test_outputs > th).int())
                test_tp += ((test_labels==1) & (pred_test_labels == 1)).sum()
                test_fp += ((test_labels==0) & (pred_test_labels == 1)).sum()
                test_fn += ((test_labels==1) & (pred_test_labels == 0)).sum()
                test_precision = test_tp / (test_tp + test_fp)
                test_recall = test_tp / (test_tp + test_fn)
                test_f_measure = 2*(test_precision*test_recall)/ (test_precision + test_recall)

            test_loss_history.append(test_loss.item())
            print(f'Test loss: {test_loss:.3f}.' \
                  #f'Test Precision: {test_recall:.3f}. ' \
                  #f'Test Recall: {test_recall:.3f}. ' \
                  f'Test F-measure: {test_f_measure:.3f}', end='. ')
            print('\n')
        
        model.train()
        
print('Training is finished!')

Epoch [1/5]. Step [1/213]. Loss: 0.673. F-measure: nan. Test loss: 0.477.Test F-measure: nan. 

Epoch [1/5]. Step [151/213]. Loss: 0.202. F-measure: 0.085. Test loss: 0.044.Test F-measure: 0.187. 

Epoch [2/5]. Step [1/213]. Loss: 0.162. F-measure: 0.471. Test loss: 0.041.Test F-measure: 0.491. 

Epoch [2/5]. Step [151/213]. Loss: 0.125. F-measure: 0.582. Test loss: 0.103.Test F-measure: 0.528. 

Epoch [3/5]. Step [1/213]. Loss: 0.128. F-measure: 0.706. Test loss: 0.006.Test F-measure: 0.591. 

Epoch [3/5]. Step [151/213]. Loss: 0.140. F-measure: 0.696. Test loss: 0.002.Test F-measure: 0.617. 

Epoch [4/5]. Step [1/213]. Loss: 0.093. F-measure: 0.667. Test loss: 0.249.Test F-measure: 0.606. 

Epoch [4/5]. Step [151/213]. Loss: 0.113. F-measure: 0.763. Test loss: 0.015.Test F-measure: 0.601. 

Epoch [5/5]. Step [1/213]. Loss: 0.063. F-measure: 0.824. Test loss: 0.019.Test F-measure: 0.591. 

Epoch [5/5]. Step [151/213]. Loss: 0.101. F-measure: 0.791. Test loss: 0.004.Test F-measure: 0.6

In [179]:
gru_2_layers_test_f_measure = test_f_measure
gru_2_layers_train_f_measure = f_measure
gru_2_layers_test_f_measure, gru_2_layers_train_f_measure

(tensor(0.6004), tensor(0.7820))

Хороший результат по метрике, но снова сильное переобучение

Попробуем ту же архитектуру, но будем усреднять скрытые состояния

In [180]:
class GRUFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=False):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        gru_out, ht = self.gru(x)
       
        if self.use_last:
            last_tensor = gru_out[:,-1,:]
        else:
            # use mean
            last_tensor = torch.mean(gru_out[:,:], dim=1)
    
        out = self.linear(last_tensor)
        return torch.sigmoid(out)

In [181]:
model = LSTMFixedLen(vocab_size=max_words)
device = 'cpu'

In [182]:
print(model)
print("Parameters:", sum([param.nelement() for param in model.parameters()]))

LSTMFixedLen(
  (embeddings): Embedding(2500, 128, padding_idx=0)
  (lstm): LSTM(128, 128, num_layers=2, batch_first=True)
  (linear): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Parameters: 584321


In [183]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [184]:
model = model.to(device)
model.train()
th = 0.5
epochs = 5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    tp, fp, fn = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()

        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        tp += ((labels==1) & (pred_labels == 1)).sum()
        fp += ((labels==0) & (pred_labels == 1)).sum()
        fn += ((labels==1) & (pred_labels == 0)).sum()
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f_measure = 2*(precision*recall)/ (precision + recall)
        #tp_running_items += len([l for l in labels if l==1])       
        # выводим статистику о процессе обучения
        if i % 150 == 0:    # печатаем каждые 150 batches
            model.eval()
            
            print(f'Epoch [{epoch + 1}/{epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {loss:.3f}. ' \
                  #f'Precision: {precision:.3f}. ' \
                  #f'Recall: {recall:.3f}. ' \
                  f'F-measure: {f_measure:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0
            train_loss_history.append(loss)

            # выводим статистику на тестовых данных
            test_tp, test_fp, test_fn = 0.0, 0.0, 0.0
            for j, data in enumerate(val_loader):
                test_labels = data[1].to(device)
                test_outputs = model(data[0].to(device))
                
                # подсчет ошибки на тесте
                test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
                # подсчет метрики на тесте

                pred_test_labels = torch.squeeze((test_outputs > th).int())
                test_tp += ((test_labels==1) & (pred_test_labels == 1)).sum()
                test_fp += ((test_labels==0) & (pred_test_labels == 1)).sum()
                test_fn += ((test_labels==1) & (pred_test_labels == 0)).sum()
                test_precision = test_tp / (test_tp + test_fp)
                test_recall = test_tp / (test_tp + test_fn)
                test_f_measure = 2*(test_precision*test_recall)/ (test_precision + test_recall)

            test_loss_history.append(test_loss.item())
            print(f'Test loss: {test_loss:.3f}.' \
                  #f'Test Precision: {test_recall:.3f}. ' \
                  #f'Test Recall: {test_recall:.3f}. ' \
                  f'Test F-measure: {test_f_measure:.3f}', end='. ')
            print('\n')
        
        model.train()
        
print('Training is finished!')

Epoch [1/5]. Step [1/213]. Loss: 0.715. F-measure: 0.131. Test loss: 0.504.Test F-measure: nan. 

Epoch [1/5]. Step [151/213]. Loss: 0.074. F-measure: 0.294. Test loss: 0.061.Test F-measure: 0.512. 

Epoch [2/5]. Step [1/213]. Loss: 0.146. F-measure: 0.667. Test loss: 0.036.Test F-measure: 0.540. 

Epoch [2/5]. Step [151/213]. Loss: 0.149. F-measure: 0.634. Test loss: 0.536.Test F-measure: 0.625. 

Epoch [3/5]. Step [1/213]. Loss: 0.103. F-measure: 0.818. Test loss: 0.129.Test F-measure: 0.604. 

Epoch [3/5]. Step [151/213]. Loss: 0.115. F-measure: 0.717. Test loss: 0.234.Test F-measure: 0.615. 

Epoch [4/5]. Step [1/213]. Loss: 0.080. F-measure: 0.727. Test loss: 0.051.Test F-measure: 0.627. 

Epoch [4/5]. Step [151/213]. Loss: 0.089. F-measure: 0.773. Test loss: 0.024.Test F-measure: 0.584. 

Epoch [5/5]. Step [1/213]. Loss: 0.044. F-measure: 0.800. Test loss: 0.012.Test F-measure: 0.624. 

Epoch [5/5]. Step [151/213]. Loss: 0.062. F-measure: 0.797. Test loss: 0.012.Test F-measure: 0

In [185]:
gru_2_layers_test_f_measure_mean_last_tensor = test_f_measure
gru_2_layers_train_f_measure_mean_last_tensor = f_measure
gru_2_layers_test_f_measure_mean_last_tensor, gru_2_layers_train_f_measure_mean_last_tensor

(tensor(0.6162), tensor(0.7946))

Лучший результат для валидационной выборки и обучается очень быстро. Но все то же переобучение.

Сделаем таблицу с результатами на валидационной выборке

In [194]:
test_result = np.array([lstm_2_layers_test_f_measure, lstm_2_layers_test_f_measure_10ep, lstm_1_layer_test_f_measure, lstm_3_layers_test_f_measure, 
                        lstm_1_layer_test_f_measure_256_emb, lstm_2_layers_test_f_measure_mean_last_tensor, 
                        gru_2_layers_test_f_measure, gru_2_layers_test_f_measure_mean_last_tensor])
naming = ['LSTM 2 слоя - 5 эпох', 'LSTM 2 слоя - 10 эпох', 'LSTM 1 слой - 5 эпох', 'LSTM 3 слоя - 5 эпох', 'LSTM 1 слой, размерность эмб и скрытого - 256',
         'LSTM 2 слоя - усреднение скрытых состояний', 'GRU 2 слоя - последнее скрытое состояние', 'GRU 2 слоя - усреднение скрытых состояний']

In [196]:
df = pd.Series(test_result, index = naming)
df

LSTM 2 слоя - 5 эпох                             0.521561
LSTM 2 слоя - 10 эпох                            0.558233
LSTM 1 слой - 5 эпох                             0.466531
LSTM 3 слоя - 5 эпох                             0.357143
LSTM 1 слой, размерность эмб и скрытого - 256    0.477733
LSTM 2 слоя - усреднение скрытых состояний       0.610568
GRU 2 слоя - последнее скрытое состояние         0.600355
GRU 2 слоя - усреднение скрытых состояний        0.616197
dtype: float32